In [18]:
pip install torch-geometric


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.9 MB/s eta 0:00:00:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.7/387.7 kB 9.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 3.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [33]:
pip install torch torchvision torchaudio



[notice] A new release of pip is available: 23.0 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [29]:
import numpy as np
import torch
from torch_geometric.data import Data
from ase.geometry import analysis
import pickle
from ase.db import connect 
from ase.neighborlist import natural_cutoffs
from ase.neighborlist import NeighborList 
from ase.visualize import view

adsorbates = ['CO','OH','OCH3','MC']
surface_elements = ['Ag', 'Au', 'Cu', 'Pd', 'Pt', 'Zn']
adsorbate_elements = ['C']
n_neighbors = 2
db_name = 'CO/CO_out_top.db'
adsorbate = 'CO'


def db_to_graphs(surface_elements, adsorbate_elements, n_neighbors, db_name):
    counter = 0
    n_rejected = 0
    graph_list = []
    # iterate through the desired database entries
    with connect(db_name) as db:
        total_rows = sum(1 for _ in db.select(relaxed=1))
        #print(f"Total rows returned by query: {total_rows}")
        for row in db.select():
            counter += 1
            #print(f'Processing row {counter}...')

            
            atoms = db.get_atoms(row.id)
            
            
            #Delete atoms from the Atoms object to make one atom in ads
            indices_to_delete = [46]
            for idx in indices_to_delete:
                del atoms[idx]
            #view(atoms)
            #stop
                
            
            # set tags of atoms from OCP that match DFT 
            tags = [5]*9 + [4]*9 + [3]*9 + [2]*9 + [1]*9
            ads_tags = [0]*(len(atoms)-len(tags))
            tags+= ads_tags
            
            atoms.set_tags(tags)
            

            # get atoms object and repeat
            
            atoms_3x3 = atoms.repeat((3, 3, 1))
            labels_3x3 = atoms_3x3.get_chemical_symbols()
            
            adj_energy = row.adj_energy

            cutoffs = natural_cutoffs(atoms_3x3)
            neighbor_list = NeighborList(cutoffs, self_interaction=False, bothways=True)
            neighbor_list.update(atoms_3x3)
            
            # get all bonds/edges in the atoms object
            ana_object = analysis.Analysis(atoms_3x3, nl=neighbor_list)
            all_edges = np.c_[np.array(list(ana_object.adjacency_matrix[0].keys()), dtype=np.dtype('int,int'))['f0'],
                              np.array(list(ana_object.adjacency_matrix[0].keys()), dtype=np.dtype('int,int'))['f1']]

            # remove self-to-self edges
            all_edges = all_edges[all_edges[:, 0] != all_edges[:, 1]]

            # adsorbing atom symbol (interpreted as the first adsorbate atom in the list of atoms)
            n_ads_atoms = len([atom.index for atom in atoms if atom.tag == 0])
            ads_atoms_ids = [atom.index for atom in atoms_3x3 if atom.tag == 0]
    
            # ids of adsorbate in the middle of the 3x3 atoms object
            ads_ids = ads_atoms_ids[n_ads_atoms * 4:n_ads_atoms * 5]

            # find all unique atom ids included in adsorbate, bond to surface, first neighbors and second neighbors
            ads_edges = all_edges[np.isin(all_edges[:, 0], ads_ids)]
            neighbor_edges = all_edges[np.isin(all_edges[:, 0], ads_edges[:, 1])]
            nextneighbor_edges = all_edges[np.isin(all_edges[:, 0], neighbor_edges[:, 1])]

            ens_ids = [id for id in np.unique(ads_edges) if id not in ads_ids]
            n_ids = [id for id in np.unique(neighbor_edges) if id not in ens_ids + ads_ids]
            nn_ids = [id for id in np.unique(nextneighbor_edges) if id not in ens_ids + ads_ids + n_ids]

            aoi = []
            for nn in nn_ids:
                n = n_ids[np.argmin(atoms_3x3.get_distances(nn, [n_ids]))]
                e = ens_ids[np.argmin(atoms_3x3.get_distances(nn, [ens_ids]))]
                a = atoms_3x3.get_angle(e, n, nn)
                if a > 150:
                    aoi.append(nn)

        
            # include nodes according to rank of neighbors
            if n_neighbors == 0:
                incl_ids = np.array(ads_ids + ens_ids)
            elif n_neighbors == 1:
                incl_ids = np.array(ads_ids + ens_ids + n_ids)
            elif n_neighbors == 2:
                incl_ids = np.array(ads_ids + ens_ids + n_ids + nn_ids)

            # shuffle the node list
            np.random.shuffle(incl_ids)
            #print('Included IDs:', incl_ids)

            # isolate all edges between included atom ids (both ways / undirected)
            all_edges_reduced = all_edges[np.all(np.isin(all_edges[:, :], incl_ids), axis=1)]
            #print('All Edges Reduced:', all_edges_reduced)

            # node feature lists (element label and onehot indexed)
            elements = surface_elements + adsorbates
            
            labels_3x3 = np.array(labels_3x3, dtype='object')
            labels_3x3[labels_3x3==adsorbate_elements[0]] = adsorbate
            #print(labels_3x3)
            #print(elements)
            # onehot encoding of the node list
            node_onehot = np.zeros((len(incl_ids), len(elements) + 2))
            for i, id in enumerate(incl_ids):
                node_onehot[i, elements.index(labels_3x3[id])] = 1
                node_onehot[i, -2] = atoms_3x3[id].tag

                if id in aoi:
                    node_onehot[i, -1] = 1

            # rename all atom ids to the index in the node list
            for i, edge in enumerate(all_edges_reduced):
                for j, id in enumerate(edge):
                    all_edges_reduced[i, j] = np.where(incl_ids == id)[0][0]
            

            

            # make torch data object
            torch_edges = torch.tensor(np.transpose(all_edges_reduced), dtype=torch.long)
            torch_nodes = torch.tensor(node_onehot, dtype=torch.float)

            # get ensemble (surface elements directly involved in the bond)
            ensemble = np.array(labels_3x3)[[id for id in ens_ids]]
            ensemble = {el: sum(ensemble == el) for el in surface_elements}

            graph_list.append(Data(x=torch_nodes, edge_index=torch_edges, y=torch.tensor([adj_energy], dtype=torch.float),
                                   ads=row.ads, ens=ensemble))
            
            
        #print(f'Rejected {n_rejected} slabs in the graph construction process.')
        return graph_list

#print(f'Performing graph construction of OH')
samples = db_to_graphs(surface_elements, adsorbate_elements, n_neighbors, db_name)
#print(f'Constructed {len(samples)} graphs.')

# Save the constructed graphs to a file using pickle
with open(f'COsamlet.graphs', 'wb') as output:
    pickle.dump(samples, output)

print('Graphs saved to graphs')


Graphs saved to graphs


In [27]:
import pickle

# Define the number of graphs to load
num_graphs_to_load = 100  # Adjust this number as needed

# Load the .graph file
with open('OHsamlet.graphs', 'rb') as f:
    #loaded_graphs = []
    #for _ in range(num_graphs_to_load):
    #    try:
    #        graph = pickle.load(f)
    #        loaded_graphs.append(graph)
    #    except EOFError:
    #        # Reached end of file
    #        break
    loaded_graphs = pickle.load(f)

# Now loaded_graphs contains a list of the first num_graphs_to_load graphs


In [28]:
# Print the loaded graphs
for i, graph in enumerate(loaded_graphs):
    print(f"Graph {i+1}:")
    print(graph)

Graph 1:
Data(
  x=[38, 12],
  edge_index=[2, 260],
  y=[1],
  ads='OH',
  ens={
    Ag=0,
    Au=0,
    Cu=0,
    Pd=1,
    Pt=0,
    Zn=0,
  }
)
Graph 2:
Data(
  x=[38, 12],
  edge_index=[2, 260],
  y=[1],
  ads='OH',
  ens={
    Ag=1,
    Au=0,
    Cu=0,
    Pd=0,
    Pt=0,
    Zn=0,
  }
)
Graph 3:
Data(
  x=[38, 12],
  edge_index=[2, 260],
  y=[1],
  ads='OH',
  ens={
    Ag=0,
    Au=0,
    Cu=0,
    Pd=1,
    Pt=0,
    Zn=0,
  }
)
Graph 4:
Data(
  x=[38, 12],
  edge_index=[2, 260],
  y=[1],
  ads='OH',
  ens={
    Ag=0,
    Au=0,
    Cu=0,
    Pd=1,
    Pt=0,
    Zn=0,
  }
)
Graph 5:
Data(
  x=[38, 12],
  edge_index=[2, 260],
  y=[1],
  ads='OH',
  ens={
    Ag=0,
    Au=1,
    Cu=0,
    Pd=0,
    Pt=0,
    Zn=0,
  }
)
Graph 6:
Data(
  x=[38, 12],
  edge_index=[2, 260],
  y=[1],
  ads='OH',
  ens={
    Ag=0,
    Au=1,
    Cu=0,
    Pd=0,
    Pt=0,
    Zn=0,
  }
)
Graph 7:
Data(
  x=[38, 12],
  edge_index=[2, 260],
  y=[1],
  ads='OH',
  ens={
    Ag=0,
    Au=1,
    Cu=0,
    Pd=

In [23]:
loaded_graphs[0].x

tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 3., 1.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 3., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 2., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 3., 1.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 2., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 3., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 2., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 3., 1.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 2., 0.],
        [0., 0

In [38]:
with connect('OH/OH_out_top.db') as db:
    for row in db.select():
        energy = row.adj_energy
        print(energy)

1.0633605495095253
1.5386928021907806
1.0759198665618896
0.9947690218687057
1.3841231018304825
1.4249414578080177
1.4030448570847511
0.9780016168951988
1.4939969144761562
1.4202426075935364
0.8178404793143272
1.265302050858736
1.3857761174440384
0.9522632658481598
0.7747900187969208
0.9684588611125946
0.9735984206199646
0.8667128682136536
0.8206969797611237
1.1052703559398651
0.7475416660308838
1.3108208402991295
0.7605615854263306
0.8732464760541916
0.8489728309214115
1.4208134450018406
0.9745174571871758
0.9764569662511349
1.0490700453519821
1.165255792438984
0.861745547503233
1.0804158598184586
0.7785714995115995
0.7714550103992224
1.0802193228155375
1.155949853360653
1.5328167118132114
0.7468342334032059
1.0985786318778992
1.389137640595436
1.2644531056284904
0.7512428984045982
1.4209526106715202
1.374939225614071
1.3411574438214302
1.3601094372570515
0.8940819203853607
0.8933431580662727
0.8379180356860161
0.891210064291954
1.180417038500309
0.9129799082875252
0.6867647673934698
1

In [ ]:
import numpy as np
import torch
from torch_geometric.data import Data
from ase.geometry import analysis
import pickle
from ase.db import connect 
from ase.neighborlist import natural_cutoffs
from ase.neighborlist import NeighborList 
from ase.visualize import view

surface_elements = ['Ag', 'Au', 'Cu', 'Pd', 'Pt', 'Zn']
adsorbate = ['C']
n_neighbors = 1
db_name = 'CO/CO_out_top.db'

def get_graph(self, coords, adsorbate, site):
        """Construct graph feature of requested site

        NB! Graphs use torch edge-templates from adjacency matrix of ASE model system.
        Hence site_ids are listed in the order matching the edge-template and will result in
        mismatch between node-list and edge-list if changed.

        Coordinates are structured as (row,coloumn,layer) with surface layer being 0, subsurface 1 etc."""

        # Get ordered list of coordinates of atoms included in graph
        if self.facet == 'fcc111':
            if site == 'ontop':
                if self.n_neighbors == 0:
                    site_ids = np.array([(0, 0, 0)])
                elif self.n_neighbors == 1:
                    site_ids = np.array([(-1, 0, 1),
                                         (-1, 1, 1),
                                         (0, 0, 1),
                                         (-1, 0, 0),
                                         (-1, 1, 0),
                                         (0, -1, 0),
                                         (0, 0, 0),
                                         (0, 1, 0),
                                         (1, -1, 0),
                                         (1, 0, 0)
                                         ])
                elif self.n_neighbors == 2:
                    site_ids = np.array([(-1, -1, 2),  #aoi
                                         (-1, 0, 2),
                                         (-1, 1, 2),  #aoi
                                         (0, -1, 2),
                                         (0, 0, 2),
                                         (1, -1, 2),  #aoi
                                         (-2, 0, 1),
                                         (-2, 1, 1),
                                         (-2, 2, 1),
                                         (-1, -1, 1),
                                         (-1, 0, 1),
                                         (-1, 1, 1),
                                         (-1, 2, 1),
                                         (0, -1, 1),
                                         (0, 0, 1),
                                         (0, 1, 1),
                                         (1, -1, 1),
                                         (1, 0, 1),
                                         (-2, 0, 0),  #aoi
                                         (-2, 1, 0),
                                         (-2, 2, 0),  #aoi
                                         (-1, -1, 0),
                                         (-1, 0, 0),
                                         (-1, 1, 0),
                                         (-1, 2, 0),
                                         (0, -2, 0),  #aoi
                                         (0, -1, 0),
                                         (0, 0, 0),
                                         (0, 1, 0),
                                         (0, 2, 0),  #aoi
                                         (1, -2, 0),
                                         (1, -1, 0),
                                         (1, 0, 0),
                                         (1, 1, 0),
                                         (2, -2, 0),  #aoi
                                         (2, -1, 0),
                                         (2, 0, 0)  #aoi
                                         ])
                    aoi_ids = [0, 2, 5, 18, 20, 25, 29, 34, 36]



        # Get ordered list of element labels of atoms in graph
        site_labels = self.grid[tuple(((site_ids + coords) % [*self.size, self.n_layers + 1]).T)]

        # Onehot encoding of elements in nodelist taking the possible ads atoms into account.
        node_onehot = np.zeros((len(site_labels) + len(adsorbate), len(self.all_elem) + 2))
        for i, label in enumerate(site_labels):
            node_onehot[i, label] = 1
            node_onehot[i, -2] = site_ids[i][2] + 1
            if self.n_neighbors == 2:
                if i in aoi_ids:
                    node_onehot[i, -1] = 1

            # Append ads atoms to the node list
        ### THIS IS A WEAK POINT. Make sure that ads atoms are added in correct order that matches edge-template!
        for i, atom in enumerate(adsorbate[::-1]):
            node_onehot[-(i + 1), self.all_elem.index(atom)] = 1

        # Initiate torch Data object
        torch_nodes = torch.tensor(node_onehot, dtype=torch.float)
        site_graph = Data(x=torch_nodes, edge_index=self.edge_dict[(adsorbate, site)], site=site, ads=adsorbate)

        return site_graph
